In [22]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

'pip' is not recognized as an internal or external command,
operable program or batch file.


2.17.0


'pip' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [24]:
train_data = pd.read_csv(train_file_path, sep="\t", names=["class", "text"])
test_data = pd.read_csv(test_file_path, sep="\t", names=["class", "text"])

In [25]:
train_data['class'].unique()

array(['ham', 'spam'], dtype=object)

In [ ]:
train_data['label'] = train_data['class'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['class'].map({'ham': 0, 'spam': 1})

train_texts = train_data['text'].astype(str)
train_labels = train_data['label'].values

test_texts = test_data['text'].astype(str)
test_labels = test_data['label'].values

In [27]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_texts, test_labels))
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [28]:
VOCAB_SIZE = 5000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)

encoder.adapt(train_dataset.map(lambda text, label: text))

In [29]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [30]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.8632 - loss: 0.6756 - val_accuracy: 0.8657 - val_loss: 0.5818
Epoch 2/10
 5/66 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.8798 - loss: 0.5754

d:\python\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8702 - loss: 0.5190 - val_accuracy: 0.8657 - val_loss: 0.3782
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8716 - loss: 0.3275 - val_accuracy: 0.8657 - val_loss: 0.2373
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8673 - loss: 0.2000 - val_accuracy: 0.8657 - val_loss: 0.1645
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8875 - loss: 0.1461 - val_accuracy: 0.9626 - val_loss: 0.1176
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.9792 - loss: 0.0897 - val_accuracy: 0.9777 - val_loss: 0.0807
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.9888 - loss: 0.0650 - val_accuracy: 0.9842 - val_loss: 0.0613
Epoch 8/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.9896 - loss: 0.0439 - val_accuracy: 0.9849 - val_loss: 0.0554
Epoch 9/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.9911 - loss: 0.0330 - val_accuracy: 0.9856 - val_loss: 0.

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_text = tf.convert_to_tensor([pred_text])
    prob_spam = float(model(pred_text, training=False))
    if prob_spam >= 0.5:
        label = "spam"
        prob = prob_spam
    else:
        label = "ham"
        prob = 1 - prob_spam
    
    return [prob, label]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


[6.495545864105225, 'ham']


In [32]:
sample_text = ("our new mobile video service is live. just install on your phone to start watching.")
predict_message(sample_text)

[2.9767117500305176, 'spam']

In [33]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
